# This code is intended to be used for offline analysis of the seismic data retreived from the WebDAQ during site surveying.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interact, IntSlider
from ipywidgets import FloatSlider
from ipywidgets import Layout 
from scipy import signal
from matplotlib import gridspec
from scipy.fft import fftshift
import warnings

################################################################################################################################
################################## This will be for the data you collect with the seismometer ##################################
####################### You'll have to remove the metadata from the csv file in order to run the script ########################
################################################################################################################################

################################### If you are using Mac, remove the r before the path #########################################
seis = pd.read_csv(r"Put\data\path\here", delimiter=',') 

sam = seis['Sample']
tim = seis['Time (s)']
noi = seis['Noise (V)']
z = seis['Channel Z (V)']
n = seis['Channel N (V)']
e = seis['Channel E (V)']

########################################### This is the sample rate we used in testing #########################################
fs = 1651.612903
############################################ If you used something else, put it here ###########################################

## The following cell displays the time series of the recorded data

In [ ]:
plt.figure(figsize = (19, 11))

####################### You can change the linestyle/order in which they are ploted for ease of viewing ########################

plt.plot(tim,z, linewidth = 1.5, color = 'black', label = 'Z') 
plt.plot(tim,n, linewidth = 1.5, color = 'red', label = 'N')
plt.plot(tim,e, linewidth = 1.5, color = 'mediumblue', label = 'E')
#plt.plot(tim,noi, linewidth = 1.5, color = 'yellow', label = 'Noise')

plt.legend(loc = "upper right", fontsize = 18)
plt.title("Seismic Data Time Series", fontweight = 'bold', fontsize = 25)
plt.xlabel("Time (s)", fontweight = "bold", fontsize = 20)
plt.ylabel("Amplitude (V)", fontweight = "bold", fontsize = 20)
plt.xlim(0, tim[len(tim)-1])
#plt.ylim()
plt.yticks(fontsize = 20, fontweight = "bold")
plt.xticks(fontsize = 20, fontweight = "bold")
plt.grid(True)

#.savefig('**place_img_name_here.format**', bbox_inches='tight')

## If you want to look at a specific time along the time series, you can use the slider plot below. Beware, there is a slight time delay

In [ ]:
Min = min(z.min(), n.min(), e.min())  ########################## This is to find the window height #############################
Max = max(z.max(), n.max(), e.max())

##################################### This function is what makes the slider run ###############################################
######################################### Parameters can be changed as needed ##################################################

def plot_with_slider(xlim):
    
    plt.figure(figsize=(19, 8))   ## dpi breaks slider
    
    plt.plot(tim, z, linewidth = 1.5, label = 'Z', color = 'black')
    plt.plot(tim, n, linewidth = 1.5, label = 'N', color = 'red')
    plt.plot(tim, e, linewidth = 1.5, label = 'E', color = 'blue')
    
    plt.xlim(xlim, xlim + 15) ######################## This is how long the window is (CAN be changed) #########################
    plt.ylim(Min - 1, Max + 1) ################## This adds a buffer to the window height (CAN be changed) #####################
    
    plt.legend(loc = 'upper right', fontsize = 15)
    plt.title("Electromagentic Data", fontweight = 'bold', fontsize = 25)
    plt.xlabel("Time (s)", fontweight = "bold", fontsize = 20)
    plt.ylabel("Amplitude (V)", fontweight = "bold", fontsize = 20)
    plt.yticks(fontsize = 20, fontweight = "bold")
    plt.xticks(fontsize = 20, fontweight = "bold")
    plt.grid(True)
    plt.show()

######################################## This is where the slider plot gets plotted ############################################
#### min,max = start,end ; step is the increment it slides on ; value is where it starts ; layout is how long the slider is ####
############################################### step and layout CAN be changed #################################################

interact(plot_with_slider, xlim = FloatSlider(min = 0, max = (tim[len(tim)-1]), step = 4, value = 0, 
                                              layout=Layout(width='980px')))


## The following cell plots spectrograms for each direction

In [ ]:
################################################################################################################################
###################### The spectrograms are calculated with signal.spectrogram from SciPy ######################################
################################################################################################################################
warnings.simplefilter('ignore')

fz, tz, Sxxz = signal.spectrogram(z, fs, window= 'hamming', nperseg=round(fs *2), nfft= round(fs *10), noverlap= round(fs / 2))
fn, tn, Sxxn = signal.spectrogram(n, fs, window= 'hamming', nperseg=round(fs *2), nfft= round(fs *10), noverlap= round(fs / 2))
fe, te, Sxxe = signal.spectrogram(e, fs, window= 'hamming', nperseg=round(fs *2), nfft= round(fs *10), noverlap= round(fs / 2))
f, t, Sxx = signal.spectrogram(noi, fs, window= 'hamming', nperseg=round(fs *2), nfft= round(fs *10), noverlap= round(fs / 2))


fig = plt.figure(figsize = (20,24))

gs = gridspec.GridSpec(4,1, height_ratios = [1,1,1,1], hspace = 0.35)

####################################################### Channel Z ##############################################################

axis1 = fig.add_subplot(gs[0,0])

axis1.set_title("Z Direction", fontweight = 'bold' , fontsize = 20)

m0 = axis1.pcolormesh(tz, fz, 10 * np.log10(Sxxz), shading='gouraud', vmin = -93, vmax = -65)

axis1.set_xlabel('Time (s)')
axis1.set_ylabel('Frequency (Hz)')
fig .colorbar(m0, label='Intensity [dB]')
axis1.set_ylim(0,20)


####################################################### Channel N ##############################################################

axis2 = fig.add_subplot(gs[1,0])

axis2.set_title("Y Direction", fontweight = 'bold' , fontsize = 20)

m1 = axis2.pcolormesh(tn, fn, 10 * np.log10(Sxxn), shading='gouraud', vmin = -93, vmax = -65)

axis2.set_xlabel('Time (s)')
axis2.set_ylabel('Frequency (Hz)')
fig.colorbar(m1, label='Intensity [dB]')
axis2.set_ylim(0,20)

####################################################### Channel E ##############################################################

axis3 = fig.add_subplot(gs[2,0])

axis3.set_title("X Direction", fontweight = 'bold' , fontsize = 20)

m2 = axis3.pcolormesh(te, fe, 10 * np.log10(Sxxi), shading='gouraud', vmin = -93, vmax = -65)

axis3.set_xlabel('Time (s)', fontweight = 'bold' , fontsize = 18)
axis3.set_ylabel('Frequency (Hz)', fontweight = 'bold' , fontsize = 18)
fig.colorbar(m2, label='Intensity [dB]')
axis3.set_ylim(0,20)


###################################################### Noise Channel ###########################################################
'''
axis4 = fig.add_subplot(gs[3,0])

axis4.set_title("Noise Direction", fontweight = 'bold' , fontsize = 20)

m3 = axis4.pcolormesh(t, f, Sxx, shading='gouraud', vmin = -93, vmax = -75)

axis4.set_xlabel('Time (s)')
axis4.set_ylabel('Frequency (Hz)')
fig.colorbar(m3, label='Intensity [dB]')
axis4.set_.ylim(0,20)

plt.savefig('**place_img_name_here.format**', bbox_inches='tight')
'''
################################################################################################################################
################################## These will take a while to plot, since they are so dense ####################################

## If you only want to look at spectra of specific parts of the time series, you can set the time limits here

In [ ]:
################################## Enter how the start and end times you want to look at #######################################
start = 0  ##in terms of seconds 
end = 100  ## You'll want to use at least a 100

d = 0
f = 0
################# If you want to look at the whole time series you can change the variables in the next cell ###################
################################################################################################################################

for d in range(0, len(tim)): ################ These are to find the index values of start,end from the data file ###############
    if round(tim[d]) == start:
        gamma = sam[d]
        break
        
for f in range(0, len(tim)):
    if round(tim[f]) == end:
        delta = sam[f]
        break

################################################### The data gets sliced #######################################################

t1 = tim[gamma:delta]
#no = noi[gamma:delta]
z1 = z[gamma:delta]
n1 = n[gamma:delta]
e1 = e[gamma:delta]

## This cell is where the PSD is calculated and plotted

In [ ]:
################################################################################################################################
######################################## The PSD is calculated via welch from scipy ############################################
################################################################################################################################
warnings.simplefilter('ignore')

############ If you don't want to look at the whole time series, you can add '1' to the variables, i.e k1, j1, i1 ##############

f_z, Pxx_den_z = signal.welch(z, fs, window= 'hamming', nperseg= (fs *2), nfft= (fs *100), noverlap= round(fs /2) )
f_n, Pxx_den_n = signal.welch(n, fs, window= 'hamming', nperseg= (fs *2), nfft= (fs *100), noverlap= round(fs /2) )
f_e, Pxx_den_e = signal.welch(e, fs, window= 'hamming', nperseg= (fs *2), nfft= (fs *100), noverlap= round(fs /2) )

################### nfft is how long the fft bin lengths, noverlap is bin overlap, window is the window used ###################
####################### We used 100 sec bins, 0.5 sec overlap, and hamming window, these can be changed ########################
########################################### # DO NOT CHANGE/REMOVE 'mag_rate' # ################################################
                    
log_z = np.log(Pxx_den_z)
log_n = np.log(Pxx_den_n) ############ The log of each PSD must be found in order to find the peaks/frequences #################
log_e = np.log(Pxx_den_e)

peak_z, _ = signal.find_peaks(log_z, prominence = 1.2) ############## This is where the frequencies are found ##################
peak_n, _ = signal.find_peaks(log_n, prominence = 1.2) ############### If some are missed or there are extra ###################
peak_e, _ = signal.find_peaks(log_e, prominence = 1.2) ########## you will need to change the prominence variable ##############


################################################################################################################################
##################################################### Plots peaks ##############################################################

plt.figure(figsize = (20, 8))#, dpi = 2500)

plt.scatter(f_z[peak_z], Pxx_den_z[peak_z] ** 0.5, s = 100, color = 'limegreen', marker = 'x', 
            linewidths = 2.5, label = 'Signal Peaks')
plt.scatter(f_n[peak_n], Pxx_den_n[peak_n] ** 0.5, s = 100, color = 'limegreen', marker = 'x', 
            linewidths = 2.5)
plt.scatter(f_e[peak_e], Pxx_den_e[peak_e] ** 0.5, s = 100, color = 'limegreen', marker = 'x', 
            linewidths = 2.5)

###################################################### PLots data ##############################################################

plt.semilogy(f_z, Pxx_den_z ** 0.5, color = 'black', linewidth = 1.75, label = 'Z Direction')
plt.semilogy(f_n, Pxx_den_n ** 0.5, color = 'red', linewidth = 1.75, label = 'N Direction')
plt.semilogy(f_e, Pxx_den_e ** 0.5, color = 'mediumblue', linewidth = 1.75, label = 'E Direction')

plt.legend(loc = "upper right", fontsize = 14.5)
plt.title("Seismic Data PSD", fontweight = 'bold', fontsize = 25)
plt.xlabel("Frequency (Hz)", fontweight = "bold", fontsize = 20)
plt.ylabel("Amplitude (V^2/Hz)", fontweight = "bold", fontsize = 20)
plt.ylim([10e-6, 10e-4])
plt.xlim(0, 150)
plt.yticks(fontsize = 20, fontweight = "bold")
plt.xticks(fontsize = 20, fontweight = "bold")
plt.grid(True)

## The following cell will plot each of direction signal in their own respective plots
### If you don't want/need to look at them individually, you don't have to run it

In [ ]:
################################################################################################################################
############################# This gets plotted as a multipanel plot via gridspec and subplot ##################################
################################################################################################################################

fig = plt.figure(figsize = (20,24))

gs = gridspec.GridSpec(3,1, height_ratios = [1,1,1], hspace = 0.35)

axis1 = fig.add_subplot(gs[0,0])

####################################################### Channel Z ##############################################################

axis1.scatter(f_z[peak_z], Pxx_den_z[peak_z]**0.5, s = 150, color = 'limegreen', marker = 'x', 
              linewidths = 4, label = 'Signal Peaks')
axis1.semilogy(f_z, Pxx_den_z ** 0.5, color = 'black', linewidth = 2, label = 'Channel Z')

axis1.set_xlabel("Frequency (Hz)", fontweight = "bold", fontsize = 18)
axis1.set_ylabel("Amplitude (V)", fontweight = "bold", fontsize = 18)

axis1.set_ylim([10e-6, 1])
axis1.set_xlim(0,150)
axis1.grid(True)

####################################################### Channel N ##############################################################

axis2 = fig.add_subplot(gs[1,0])

axis2.scatter(f_n[peak_n], Pxx_den_n[peak_n]**0.5, s = 150, color = 'limegreen', marker = 'x', 
              linewidths = 4)
axis2.semilogy(f_n, Pxx_den_n ** 0.5, color = 'red', linewidth = 2, label = 'Channel N')

axis2.set_xlabel("Frequency (Hz)", fontweight = "bold", fontsize = 18)
axis2.set_ylabel("Amplitude (V)", fontweight = "bold", fontsize = 18)

axis2.set_ylim([10e-6, 1])
axis2.set_xlim(0,150)
axis2.grid(True)

####################################################### Channel E ##############################################################

axis3 = fig.add_subplot(gs[2,0])

axis3.scatter(f_e[peak_e], Pxx_den_e[peak_e]**0.5, s = 150, color = 'limegreen', marker = 'x', 
              linewidths = 4)
axis3.semilogy(f_e, Pxx_den_e ** 0.5, color = 'mediumblue', linewidth = 2, label = 'Channel E')

axis3.set_xlabel("Frequency (Hz)", fontweight = "bold", fontsize = 18)
axis3.set_ylabel("Amplitude (V)", fontweight = "bold", fontsize = 18)

axis3.set_ylim([10e-6, 1])
axis3.set_xlim(0,150)
axis3.grid(True)


## This cell will output all the frequencies found and their respective amplitudes into a csv file

In [ ]:
####################################################### # # # # # # ############################################################
################################################### # SPAGHETTI CODE # #########################################################
###################################################### # # # # # # # ###########################################################
################################################################################################################################
###################################### Since each direction has a different number of peaks ####################################
############################################### They cannot be printed out right ###############################################
########################################### So they have to be made the same length ############################################
################################################################################################################################

l = 0 ################################################## Variables for loops ###################################################
m = 0
o = 0
q = 0

index = [] ############################################ Empty sets to be appended to ###########################################
roll = []
inventory = []

tsukishima = max(len(f_e[peak_e]), len(f_n[peak_n]), len(f_z[peak_z])) # Finds the directions with the most peaks, i.e longest #

for q in range(0, tsukishima - len(f_e[peak_e])): ############# The shorter data sets will be made equal with 'Nan' ############
    inventory.append('NaN')                       ################## Nothing will happen to the longest set ####################
for m in range(0, tsukishima - len(f_n[peak_n])): ################ If they are the same length nothing happens #################
    index.append('NaN')
for o in range(0, tsukishima - len(f_z[peak_z])):
    roll.append('NaN')

################################################################################################################################
######################################### This path is where the file gets outputted ###########################################
####################################################### Change as need #########################################################

with open (r"C:\Users\cacam\Documents\Seis_frequencies.csv",'w') as f: 
        f.write('Frequency_E (HZ), Amplitude_E (V), Frequency_N (Hz), Amplitude_N (V), Frequency_Z (Hz), Amplitude_Z (V)\n')
        
        for l in range(0, tsukishima):
            if (len(f_e[peak_e]) != len(f_n[peak_n]) != len(f_z[peak_z])):                
                if len(f_e[peak_e]) == tsukishima:      
                    
############################################# The files are made the same length ###############################################

                    dir3 = np.append(f_n[peak_n], index)
                    dir_3 = np.append(Pxx_den_n[peak_n], index)
                    dir4 = np.append(f_z[peak_z], roll)
                    dir_4 = np.append(Pxx_den_z[peak_z], roll)

##################################### This where the dat points get printed in the file ########################################

                    f.write(str(f_e[peak_e][l]) + ',' + str(Pxx_den_e[peak_e][l]) + ',' +    
                            str(dir3[l]) + ',' + str(dir_3[l]) + ',' +
                            str(dir4[l]) + ',' + str(dir_4[l]) + '\n')
                    
###################### The elif's are for if a different direction has more peaks, longest data set ############################

                elif len(f_n[peak_n]) == tsukishima:
                    
                    dir3 = np.append(f_e[peak_e], inventory)
                    dir_3 = np.append(Pxx_den_e[peak_e], inventory)
                    dir4 = np.append(f_z[peak_z], roll)
                    dir_4 = np.append(Pxx_den_z[peak_z], roll)
                    
                    f.write(str(dir3[l]) + ',' + str(dir_3[l]) + ',' +    
                            str(f_n[peak_n][l]) + ',' + str(Pxx_den_n[peak_n][l]) + ',' +
                            str(dir4[l]) + ',' + str(dir_4[l]) + '\n')

                    
                elif len(f_z[peak_z]) == tsukishima:
                    
                    dir3 = np.append(f_e[peak_e], inventory)
                    dir_3 = np.append(Pxx_den_e[peak_e], inventory)
                    dir4 = np.append(f_n[peak_n], index)
                    dir_4 = np.append(Pxx_den_n[peak_n], index)
                    
                    f.write(str(dir3[l]) + ',' + str(dir_3[l]) + ',' +    
                            str(dir4[l]) + ',' + str(dir_4[l]) + ',' +
                            str(f_z[peak_z][l]) + ',' + str(f_z[peak_z][l]) + '\n')

############################### If the data sets are the same length the data will be printed here #############################

            else:
                f.write(str(f_e[peak_e][l]) + ',' + str(Pxx_den_e[peak_e][l]) + ',' +    
                        str(f_n[peak_n][l]) + ',' + str(Pxx_den_n[peak_n][l]) + ',' +
                        str(f_z[peak_z][l]) + ',' + str(Pxx_den_z[peak_z][l]) + '\n')
print('It ran')